In the last notebook, I found that SAM is computing a drastically wrong forcing in the boundary layer. How do we fix this? I have a couple of idea about the cause

1. The wind fields are imcompatible in the PBL, perhaps because the horizontal velocities are interface averages, but the vertical velocity is an average over a whole block.
2. The sub-grid-scale scheme makes very large errors.

Let's load and plot the training data to see, if either of these errors seems more obvious.

In [ ]:
%matplotlib inline

import xarray as xr
from os.path import join, abspath
import os, sys
from toolz import pipe
import numpy as np
import matplotlib.pyplot as plt

sys.path.insert(0, abspath("../"))
sys.path

import holoviews as hv
hv.extension('bokeh')
%opts Image[width=500, height=300, colorbar=True](cmap='magma') 
%opts QuadMesh[width=300,  height=150, colorbar=True](cmap='viridis')
%opts Curve[width=500, height=int(500/1.61)]



# PATHS
DATA = abspath(os.getenv('RUN', '../data/processed/training.nc'))
NGROOT = "/Users/noah/Data/2018-05-30-NG_5120x2560x34_4km_10s_QOBS_EQX/"
# open data
ds = xr.open_dataset(DATA)
# only need to work with first time step
ds0 = ds.isel(time=0)
ds0

In [ ]:
lay = hv.Image(ds0.V[0], label='V') + hv.Image(ds0.U[0], label='U') + hv.Image(ds0.W[1], label='W')
lay.cols(2)

What does the horizontal divergence in the first layer look like?

In [ ]:
!pip install -U git+https://github.com/nbren12/gnl@master#subdirectory=python


In [ ]:
from gnl.xarray.sam.regrid import diff_bc, shift_bc

In [ ]:
d = 160e3

def compute_div(ds0, rho):

    rhow = (rho.shift(z=1).fillna(rho[0]) + rho)/2
    rhodz = ds.layer_mass

    hdiv = (ds0.V.shift(y=-1).fillna(0.0) - ds0.V)/d + (shift_bc(ds0.U, 1, 'x', boundary='wrap') - ds0.U)/d
    hdiv.name = 'divergence'

    vdiv = ((ds0.W*rhow).shift(z=-1)-(rhow*ds0.W))/rhodz
    vdiv.name = 'Vertical Divergence'

    total_div = hdiv + vdiv
    total_div.name = 'Total Div'
    
    return hdiv, vdiv, total_div

hdiv, vdiv, total_div = compute_div(ds0, ds.rho)

In [ ]:
hv.Dataset(-hdiv[0]).to.image(["x", "y"]) + hv.Dataset(vdiv[0]).to.image(["x", "y"])

In [ ]:
%%opts Image{+framewise}
hv.Dataset(total_div).to.image(["x", "y"], dynamic=True)

The divergence is not exactly 0, to machine precision. The relative error is about 1e-3. What happens if I process these inputs with SAM?

Here is a bunch of code to process the first sample of the NGAqua data.

In [ ]:
from sam.case import get_ngqaua_ic, InitialConditionCase

ic = get_ngqaua_ic(NGROOT, 0)

def process_ic(ic, **kwargs):

    # open initial conditions
    case = InitialConditionCase(ic=ic)

    # configure the sam run
    case.prm['parameters'].update({
        'dosgs': False,
        'dowally': True,
        'nstop': 2,
        'nsave3d': 1,
        'nstat': 2,
        'nstatfrq': 1,
        'dt': 30
    })
    
    case.prm['parameters'].update(kwargs)

    case.save()

    # Run the model
    print(case.path)
    !cd {case.path} &&  bash -c './run.sh > out  2> err'
    # !cd {case.path} &&  bash -c './run.sh'

    # open the 3d outputs
    sam_processed = xr.open_mfdataset(join(case.path, 'OUT_3D', '*.nc'), concat_dim='time').load()
    assert len(sam_processed.time) > 1

    # clean up the working directory
    !rm -rf {case.path}
    
    return sam_processed


def compute_forcings(sam_processed):
    # compute FSLI
    SLI = sam_processed.SLI
    fsli = SLI.diff('time')/SLI.time.diff('time')
    fsli.name = 'FSLI'

    # compute FQT
    QT = sam_processed.QT
    FQT = QT.diff('time')/QT.time.diff('time')
    FQT.name = 'FQT'
    
    return fsli, FQT

sam_processed = process_ic(ic)
fsli, FQT = compute_forcings(sam_processed)

I ran SAM for 2 time-steps, and here I am plotting SLI(1) - SLI(0) and SLI(2)-SLI(1),  which are two estimates of the forcing. I plot both to see if the problem is unique to forcing computed on the first time step.

In [ ]:
%%opts Image {-framewise}

hv.Dataset(fsli).to.image(["x", "y"], dynamic=True).layout('time').cols(1)

In [ ]:
%%opts Image {-framewise}

hv.Dataset(FQT).to.image(["x", "y"], dynamic=True).layout('time').cols(1)

The problem is much more obvious in the temperature field than it is for humidity, but the issue is still present.

Here is the divergence field. Should this be smaller?

In [ ]:
%%opts Image{+framewise}

hdiv, vdiv, total_div = compute_div(sam_processed.isel(time=0), ds.rho)

hv.Dataset(total_div).to.image(["x", "y"], dynamic=True)

Is the problem related to SLI?

In [ ]:
%%opts Image{+framewise}

hv.Dataset(sam_processed.SLI[0,:4]).to.image(["x", "y"])

SLI looks fine. I don't see what the problem could be.

# Chris ideas

I spoke to Chris and he has some ideas about what might be causing our problem.
1. Reference density profile is off
2. some problem with lower boundary condition
3. some problem with north/south boundaries